# ETL Project
##    Team Baldies
###                   Keith Ross             
###                 Jordan Whitney
###                  Brian Ross
# Extract
## - Data Sets 
##        -Website scraping/extraction
###             - Player Salaries from http://www.espn.com/nba/salaries/
###             - Player Stats from https://www.nba.com
# Transform
### - Consistant formatting

# Load 
## - MongosDB 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from urllib.request import urlopen
import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
url = 'http://www.espn.com/nba/salaries' 

html = urlopen(url)

In [3]:
soup = BeautifulSoup(html)

In [4]:
soup.findAll('tr',limit=4)[1].findAll('td')

[<td>1</td>,
 <td><a href="http://www.espn.com/nba/player/_/id/3975/stephen-curry">Stephen Curry</a>, PG</td>,
 <td><a href="http://www.espn.com/nba/team/_/name/gs/golden-state-warriors">Golden State Warriors</a></td>,
 <td style="text-align:right;">$37,457,154</td>]

In [5]:
column_headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('td')]

In [6]:
column_headers

['1', 'Stephen Curry, PG', 'Golden State Warriors', '$37,457,154']

In [7]:
data_rows = soup.findAll('tr')[1:]

In [8]:
player_data = [[td.getText() for td in data_rows[i].findAll('td')]
              for i in range(len(data_rows))]

In [9]:
player_data_02 = []
for i in range(len(data_rows)):
    player_row = []
    for td in data_rows[i].findAll('td'):
        player_row.append(td.getText())
        
    player_data_02.append(player_row)

In [10]:
player_data == player_data_02

True

# Transform
### - Consistant formatting
### - http://www.espn.com/nba/salaries

In [21]:
nba_salary_df = pd.DataFrame(player_data, columns=column_headers)
nba_salary_df.head()

,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154"
0,1,"Stephen Curry, PG",Golden State Warriors,"$37,457,154"
1,2,"Russell Westbrook, PG",Oklahoma City Thunder,"$35,654,150"
2,3,"Chris Paul, PG",Houston Rockets,"$35,654,150"
3,4,"Blake Griffin, PF",Detroit Pistons,"$32,088,932"
4,5,"Gordon Hayward, SF",Boston Celtics,"$31,214,295"


In [11]:
nba_salary_df = pd.DataFrame(player_data, columns=column_headers)
nba_salary_df.columns = ['Index','Name', 'Team','Salary']
del nba_salary_df['Index']
del nba_salary_df['Team']
nba_salary_df_revised = nba_salary_df['Name'].str.split(',', n = 1, expand = True)
del nba_salary_df_revised[1]
merged_nba_salary_df = nba_salary_df.merge(nba_salary_df_revised, how='outer', left_index = True, right_index = True)
merged_nba_salary_df.columns = ['name','Salary','Name']
del merged_nba_salary_df['name']
merged_nba_salary_df = merged_nba_salary_df[['Name','Salary']]
merged_nba_salary_df.head()

,Name,Salary
0,Stephen Curry,"$37,457,154"
1,Russell Westbrook,"$35,654,150"
2,Chris Paul,"$35,654,150"
3,Blake Griffin,"$32,088,932"
4,Gordon Hayward,"$31,214,295"


##             - Player Stats from https://www.nba.com

In [22]:
nba_player_stats = pd.read_csv('./NBA_player_stats.csv')
nba_player_stats.head()

,#,PLAYER,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF
0,1,James Harden,78,36.8,36.1,10.8,24.5,44.2,4.8,13.2,...,11.0,87.9,0.8,5.8,6.6,7.5,2.0,0.7,5.0,33.1
1,2,Paul George,77,36.9,28.0,9.2,21.0,43.8,3.8,9.8,...,7.0,83.9,1.4,6.8,8.2,4.1,2.2,0.4,2.7,27.4
2,3,Giannis Antetokounmpo,72,32.8,27.7,10.0,17.3,57.8,0.7,2.8,...,9.5,72.9,2.2,10.3,12.5,5.9,1.3,1.5,3.7,35.3
3,4,Joel Embiid,64,33.7,27.5,9.1,18.7,48.4,1.2,4.1,...,10.1,80.4,2.5,11.1,13.6,3.7,0.7,1.9,3.5,32.2
4,5,Stephen Curry,69,33.8,27.3,9.2,19.4,47.2,5.1,11.7,...,4.2,91.6,0.7,4.7,5.3,5.2,1.3,0.4,2.8,26.1


In [13]:
nba_player_stats_df = pd.DataFrame(nba_player_stats)
nba_player_stats_df = nba_player_stats_df.drop(['#'], axis=1)
nba_player_stats_df = nba_player_stats_df.rename(columns={'PLAYER':'Name'})
nba_player_stats_df.head()

,Name,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF
0,James Harden,78,36.8,36.1,10.8,24.5,44.2,4.8,13.2,36.8,...,11.0,87.9,0.8,5.8,6.6,7.5,2.0,0.7,5.0,33.1
1,Paul George,77,36.9,28.0,9.2,21.0,43.8,3.8,9.8,38.6,...,7.0,83.9,1.4,6.8,8.2,4.1,2.2,0.4,2.7,27.4
2,Giannis Antetokounmpo,72,32.8,27.7,10.0,17.3,57.8,0.7,2.8,25.6,...,9.5,72.9,2.2,10.3,12.5,5.9,1.3,1.5,3.7,35.3
3,Joel Embiid,64,33.7,27.5,9.1,18.7,48.4,1.2,4.1,30.0,...,10.1,80.4,2.5,11.1,13.6,3.7,0.7,1.9,3.5,32.2
4,Stephen Curry,69,33.8,27.3,9.2,19.4,47.2,5.1,11.7,43.7,...,4.2,91.6,0.7,4.7,5.3,5.2,1.3,0.4,2.8,26.1


## - Merge 2 dataframes

In [14]:
merged_nba_analysis_df = merged_nba_salary_df.merge(nba_player_stats_df, how='outer', on = ['Name'])
merged_nba_analysis_df = merged_nba_analysis_df.dropna()
merged_nba_analysis_df = merged_nba_analysis_df.reset_index(drop=True)
merged_nba_analysis_df = merged_nba_analysis_df.set_index('Name')
merged_nba_analysis_df = merged_nba_analysis_df.sort_values('Salary', ascending = True)
merged_nba_analysis_df.head()    


,Salary,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF
Name,,,,,,,,,,,,,,,,,,,,,
Jimmy Butler,"$20,445,779",65.0,33.6,18.7,6.4,13.9,46.2,1.0,3.0,34.7,...,5.6,85.5,1.9,3.4,5.3,4.0,1.9,0.6,1.5,20.7
Danilo Gallinari,"$21,587,579",68.0,30.3,19.8,6.0,13.0,46.3,2.4,5.5,43.3,...,6.0,90.4,0.8,5.3,6.1,2.6,0.7,0.3,1.5,20.6
Aaron Gordon,"$21,590,909",78.0,33.8,16.0,6.0,13.4,44.9,1.6,4.4,34.9,...,3.2,73.1,1.7,5.7,7.4,3.7,0.7,0.7,2.1,18.2
Serge Ibaka,"$21,666,667",74.0,27.2,15.0,6.3,11.9,52.9,0.7,2.3,29.0,...,2.4,76.3,2.1,6.0,8.1,1.3,0.4,1.4,1.5,18.6
LaMarcus Aldridge,"$22,347,015",81.0,33.2,21.3,8.4,16.3,51.9,0.1,0.5,23.8,...,5.1,84.7,3.1,6.1,9.2,2.4,0.5,1.3,1.8,24.4


# Load 
## - MongosDB 

In [15]:
data_dict = merged_nba_analysis_df.to_dict()

In [16]:
data_dict

{'Salary': {'Jimmy Butler': '$20,445,779',
  'Danilo Gallinari': '$21,587,579',
  'Aaron Gordon': '$21,590,909',
  'Serge Ibaka': '$21,666,667',
  'LaMarcus Aldridge': '$22,347,015',
  'DeAndre Jordan': '$22,897,200',
  'Kawhi Leonard': '$23,114,067',
  'Rudy Gobert': '$23,241,573',
  'Nicolas Batum': '$24,000,000',
  'Harrison Barnes': '$24,107,258',
  'Marc Gasol': '$24,119,025',
  'Giannis Antetokounmpo': '$24,157,303',
  'Steven Adams': '$24,157,303',
  'Nikola Jokic': '$24,605,181',
  'Hassan Whiteside': '$25,434,262',
  'Bradley Beal': '$25,434,263',
  'Andrew Wiggins': '$25,467,250',
  'Joel Embiid': '$25,467,250',
  'CJ McCollum': '$25,759,766',
  'Jrue Holiday': '$25,976,111',
  'DeMar DeRozan': '$27,739,975',
  'Damian Lillard': '$27,977,689',
  'Al Horford': '$28,928,709',
  'Paul Millsap': '$29,230,769',
  'Kevin Durant': '$30,000,000',
  'James Harden': '$30,421,854',
  'Mike Conley': '$30,521,115',
  'Paul George': '$30,560,700',
  'Kyle Lowry': '$31,200,000',
  'Gordon H

In [18]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [19]:
db = client.nba_player_analysis_db
collection = db.salary
collection.insert_one(data_dict)


In [17]:
merged_nba_analysis_df = merged_nba_analysis_df.reset_index()
merged_nba_analysis_df.head()

,Name,Salary,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF
0,Jimmy Butler,"$20,445,779",65.0,33.6,18.7,6.4,13.9,46.2,1.0,3.0,...,5.6,85.5,1.9,3.4,5.3,4.0,1.9,0.6,1.5,20.7
1,Danilo Gallinari,"$21,587,579",68.0,30.3,19.8,6.0,13.0,46.3,2.4,5.5,...,6.0,90.4,0.8,5.3,6.1,2.6,0.7,0.3,1.5,20.6
2,Aaron Gordon,"$21,590,909",78.0,33.8,16.0,6.0,13.4,44.9,1.6,4.4,...,3.2,73.1,1.7,5.7,7.4,3.7,0.7,0.7,2.1,18.2
3,Serge Ibaka,"$21,666,667",74.0,27.2,15.0,6.3,11.9,52.9,0.7,2.3,...,2.4,76.3,2.1,6.0,8.1,1.3,0.4,1.4,1.5,18.6
4,LaMarcus Aldridge,"$22,347,015",81.0,33.2,21.3,8.4,16.3,51.9,0.1,0.5,...,5.1,84.7,3.1,6.1,9.2,2.4,0.5,1.3,1.8,24.4


# - Visuals

In [18]:
data = [go.Scatter(x = merged_nba_analysis_df['Salary'], y = merged_nba_analysis_df['PTS'], text = merged_nba_analysis_df['Name'], hoverinfo = 'text', marker = dict(color='darkblue'), mode = 'markers', name = 'markers', showlegend = False)]
layout = go.Layout(title='Points Per Game vs Salary of top NBA Players',xaxis=dict(title='Salary in Millions', titlefont=dict(size = 14)),yaxis=dict(title='Points Per Game', titlefont=dict(size = 14)))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='add-hover-text')

C:\Users\rossk\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



### - NBA Efficiency, invented by Martin Manley, is being considered the first ever player evaluation metric which indicates player’s linear efficiency.
### - NBA Efficiency Formula= (Points)+(Rebounds)+(Steals)+(Assists)+(Blocked Shots)-(Turnovers)-(Missed Shots/Games Played

In [19]:
data1 = [go.Scatter(x = merged_nba_analysis_df['Salary'], y = merged_nba_analysis_df['EFF'], text = merged_nba_analysis_df['Name'], hoverinfo = 'text', marker = dict(color='darkblue'), mode = 'markers', name = 'markers', showlegend = False)]
layout1 = go.Layout(title='Effeciency vs Salary of top NBA Players',xaxis=dict(title='Salary in Millions', titlefont=dict(size = 14)),yaxis=dict(title='Effeciency Rating', titlefont=dict(size = 14)))
fig1 = go.Figure(data=data1, layout=layout1)
py.iplot(fig1, filename='add-hover-text')